installing Mealpy

In [ ]:
!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 63.5 MB/s eta 0:00:00


importing Packages

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import cv2
import time
import torch.nn as nn
import matplotlib.pyplot as plt
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
import pywt
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Bidirectional, LSTM, Dense, Dropout, Reshape
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications.densenet import preprocess_input
from sklearn.metrics.pairwise import rbf_kernel
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# from tensorsflow.keras.models import load_model
# model.save('drive location to be saved and extension is .h5 or hdf5')

Connecting Drive

In [ ]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Preprocessing,Labeling

In [ ]:

# Function to read and preprocess images
def read_and_preprocess_images(image_path, target_size=(224, 224)):
    images = []
    for filename in os.listdir(image_path):
        if filename.endswith(".jpg"):
            img = cv2.imread(os.path.join(image_path, filename), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, target_size)
            img = np.stack((img,) * 3, axis=-1)  # Convert grayscale to RGB by stacking
            img = preprocess_input(img)  # Preprocess using DenseNet function
            images.append(img)
    return np.array(images)

# Load images
normal_train_images = read_and_preprocess_images('/content/drive/MyDrive/DATASET/train/normal')
tumor_train_images = read_and_preprocess_images('/content/drive/MyDrive/DATASET/train/pancreatic_tumor')
normal_test_images = read_and_preprocess_images('/content/drive/MyDrive/DATASET/test/normal')
tumor_test_images = read_and_preprocess_images('/content/drive/MyDrive/DATASET/test/pancreatic_tumor')

# Data Augmentation
train_dataset = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_images = train_dataset.flow(
    np.concatenate((normal_train_images, tumor_train_images), axis=0),
    np.concatenate((np.zeros(normal_train_images.shape[0]), np.ones(tumor_train_images.shape[0])), axis=0),
    batch_size=32)


test_images = train_dataset.flow(
    np.concatenate((normal_test_images, tumor_test_images), axis=0),
    np.concatenate((np.zeros(normal_test_images.shape[0]), np.ones(tumor_test_images.shape[0])), axis=0),
    batch_size=32)

# Create labels for the test data
train_labels =  np.concatenate((np.zeros(normal_train_images.shape[0]), np.ones(tumor_train_images.shape[0])), axis=0)
test_labels =  np.concatenate((np.zeros(normal_test_images.shape[0]), np.ones(tumor_test_images.shape[0])), axis=0)
# Create TensorFlow datasets for training and testing


In [ ]:
# Create labels for the training data
train_labels_normal = np.zeros(normal_train_images.shape[0])
train_labels_tumor = np.ones(tumor_train_images.shape[0])
print()

Feature Extraction Using Densenet121

In [ ]:
# Load the pre-trained DenseNet121 model
densenet121 = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in densenet121.layers:
    layer.trainable = False

# Define the feature extraction model
feature_extractor = tf.keras.Sequential([
    densenet121,
    GlobalAveragePooling2D()
])

# Extract features from the preprocessed and labeled CT images
train_features = feature_extractor.predict(train_images)
test_features = feature_extractor.predict(test_images)

# Flatten the features for input to classifiers
train_features_flat = train_features.reshape(train_features.shape[0], -1)
test_features_flat = test_features.reshape(test_features.shape[0], -1)


13/13 [==============================] - 8s 557ms/step


Tried for cloning HHO

In [ ]:
# !git clone https://github.com/cahitberkay/Harris-Hawks-Optimization-HHO---Python-Code.git
# import sys
# sys.path.append('Harris-Hawks-Optimization-HHO---Python-Code')
# from HHO import HHO

Creating HHO for Hyperparameter Tuning

In [ ]:
# Harris Hawks Optimization (HHO) Class
class HHO:
    def __init__(self, objective_function, search_space, pop_size=20, max_iter=50):
        self.obj_func = objective_function
        self.search_space = search_space
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.position = np.zeros((self.pop_size, len(self.search_space)))
        self.fitness = np.zeros(self.pop_size)
        self.best_position = np.zeros(len(self.search_space))
        self.best_fitness = float('inf')

    def initialize_population(self):
        for i in range(self.pop_size):
            for j in range(len(self.search_space)):
                key = list(self.search_space.keys())[j]
                # Ensure values are sampled correctly
                self.position[i, j] = random.choice(self.search_space[key])

    def update_position(self, t):
        r1 = random.random()
        r2 = random.random()
        for i in range(self.pop_size):
            if r1 >= 0.5:  # Exploration phase
                X_rand = random.choice(self.position)
                self.position[i] = self.best_position + r2 * (X_rand - self.position[i])
            else:  # Exploitation phase
                m = 2 * (1 - t / self.max_iter)
                self.position[i] = (self.best_position - self.position[i]) * np.exp(m * t) + self.best_position

            # Ensure valid hyperparameter values
            self.position[i, 0] = max(1e-5, min(self.position[i, 0], 1e-3))  # Learning rate
            self.position[i, 1] = max(0.1, min(self.position[i, 1], 0.5))    # Dropout rate
            self.position[i, 2] = max(128, min(int(self.position[i, 2]), 512))  # Dense units

            # Evaluate new positions
            self.fitness[i] = self.obj_func(dict(zip(self.search_space.keys(), self.position[i])))
            if self.fitness[i] < self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_position = self.position[i]

    def optimize(self):
        self.initialize_population()
        for t in range(self.max_iter):
            self.update_position(t)
        return self.best_position, self.best_fitness


Hyperparameter Tuning Model


In [ ]:
# Define the search space for hyperparameter tuning
search_space = {
    'learning_rate': [1e-5, 1e-4, 1e-3],  # Ensure positive values
    'dropout_rate': [0.1, 0.3, 0.5],      # Ensure values between 0 and 1
    'dense_units': [128, 256, 512]        # Ensure positive integers
}

# Define the objective function for HHO
def objective_function(params):
    learning_rate = params['learning_rate']
    dropout_rate = params['dropout_rate']
    dense_units = int(params['dense_units'])

    # Ensure dense_units is a positive integer
    if dense_units <= 0:
        return 1e9  # Penalize invalid dense_units

    # Build the model
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model and evaluate on the test set
    model.fit(train_images, epochs=5, validation_data=test_images)
    _, val_acc = model.evaluate(test_images)
    return -val_acc  # Minimize the negative validation accuracy

# Initialize the HHO optimizer
hho = HHO(objective_function, search_space, pop_size=2, max_iter=3)
best_params, best_fitness = hho.optimize()

# Create a dictionary from the best parameters
best_params_dict = dict(zip(search_space.keys(), best_params))

# Print the best hyperparameters
print("Best hyperparameters:")
print(f"Learning rate: {best_params_dict['learning_rate']}")
print(f"Dropout rate: {best_params_dict['dropout_rate']}")
print(f"Dense units: {best_params_dict['dense_units']}")

Epoch 1/5
32/32 [==============================] - 35s 926ms/step - loss: 0.7577 - accuracy: 0.5641 - val_loss: 0.5326 - val_accuracy: 0.8152
Epoch 2/5
32/32 [==============================] - 27s 827ms/step - loss: 0.6309 - accuracy: 0.6703 - val_loss: 0.4993 - val_accuracy: 0.8633
Epoch 3/5
32/32 [==============================] - 26s 812ms/step - loss: 0.5468 - accuracy: 0.7475 - val_loss: 0.4598 - val_accuracy: 0.8481
Epoch 4/5
32/32 [==============================] - 26s 820ms/step - loss: 0.4769 - accuracy: 0.8026 - val_loss: 0.4119 - val_accuracy: 0.8861
Epoch 5/5
13/13 [==============================] - 8s 560ms/step - loss: 0.3859 - accuracy: 0.9038
Epoch 1/5
32/32 [==============================] - 34s 871ms/step - loss: 0.6764 - accuracy: 0.5952 - val_loss: 0.6592 - val_accuracy: 0.5899
Epoch 2/5
32/32 [==============================] - 26s 803ms/step - loss: 0.5430 - accuracy: 0.7415 - val_loss: 0.5823 - val_accuracy: 0.7139
Epoch 3/5
32/32 [==============================] 

Creating CNN-BiLSTM model for Classification

In [ ]:


def create_cnn_bilstm_model(input_shape, num_classes, lstm_units=128, dropout_rate=0.5):
    model = Sequential()
    # CNN part
    model.add(Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(256, 3, activation='relu'))
    model.add(GlobalAveragePooling1D())
    model.add(Reshape((1, 256)))
    # BiLSTM part
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    model.add(Bidirectional(LSTM(lstm_units // 2)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Assuming DenseNet has extracted features with shape (number_of_samples, feature_dimension)

# Reshape the data for Conv1D
train_features_flat = train_features_flat.reshape(train_features_flat.shape[0], train_features_flat.shape[1], 1)
test_features_flat = test_features_flat.reshape(test_features_flat.shape[0], test_features_flat.shape[1], 1)


# Convert labels to categorical
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=2)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=2)

# Define the input shape and number of classes
input_shape = (train_features_flat.shape[1], 1)
num_classes = 2

# Create and summarize the model
model = create_cnn_bilstm_model(input_shape, num_classes)
model.summary()

  # Train the model
model.fit(train_features_flat, train_labels, epochs=46, validation_data=(test_features_flat, test_labels))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1022, 64)          256       
                                                                 
 max_pooling1d (MaxPooling1  (None, 511, 64)           0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 509, 128)          24704     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 254, 128)          0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 252, 256)          98560     
                                                                 
 global_average_pooling1d (  (None, 256)              

Parameter Optimization using the Sparrow Search Algorithm (SSA)

In [ ]:


# Define the SSA
class SSA:
    def __init__(self, objective_function, search_space, pop_size=20, max_iter=50):
        self.obj_func = objective_function
        self.search_space = search_space
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.position = np.zeros((self.pop_size, len(self.search_space)))
        self.fitness = np.zeros(self.pop_size)
        self.best_position = np.zeros(len(self.search_space))
        self.best_fitness = float('inf')

    def initialize_population(self):
        for i in range(self.pop_size):
            for j in range(len(self.search_space)):
                self.position[i, j] = random.uniform(self.search_space[j][0], self.search_space[j][1])
            self.fitness[i] = self.obj_func(self.position[i])
            if self.fitness[i] < self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_position = self.position[i]

    def update_position(self, t):
        r1 = random.random()
        r2 = random.random()
        for i in range(self.pop_size):
            if r1 < 0.5:
                self.position[i] = self.best_position + r2 * (self.position[i] - self.best_position)
            else:
                self.position[i] = self.best_position - r2 * (self.best_position - self.position[i])
            self.fitness[i] = self.obj_func(self.position[i])
            if self.fitness[i] < self.best_fitness:
                self.best_fitness = self.fitness[i]
                self.best_position = self.position[i]

    def optimize(self):
        self.initialize_population()
        for t in range(self.max_iter):
            self.update_position(t)
        return self.best_position, self.best_fitness

# Define the search space
search_space = [(1e-5, 1e-2), (0.1, 0.5), (64, 256)]
# Define the objective function for SSA
def objective_function(params):
    learning_rate, dropout_rate, lstm_units = params
    model = create_cnn_bilstm_model((1024, 1), 2, int(lstm_units), dropout_rate)  # Update the input shape
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(train_features_flat, train_labels, epochs=3, validation_data=(test_features_flat, test_labels), verbose=0)
    _, val_acc = model.evaluate(test_features_flat, test_labels, verbose=0)
    return -val_acc


# Initialize the SSA optimizer
ssa = SSA(objective_function, search_space, pop_size=2, max_iter=3)
best_params, best_fitness = ssa.optimize()

# Extract best hyperparameters
best_learning_rate, best_dropout_rate, best_lstm_units = best_params

# Train the final model with the best hyperparameters
final_model = create_cnn_bilstm_model((1024, 1), 2, int(best_lstm_units), best_dropout_rate)  # Update the input shape
final_model.compile(optimizer=Adam(learning_rate=best_learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
final_model.fit(train_features_flat, train_labels, epochs=76, validation_data=(test_features_flat, test_labels))


Epoch 1/76
32/32 [==============================] - 8s 73ms/step - loss: 0.6866 - accuracy: 0.5631 - val_loss: 0.7133 - val_accuracy: 0.4734
Epoch 2/76
32/32 [==============================] - 1s 30ms/step - loss: 0.6820 - accuracy: 0.5782 - val_loss: 0.7197 - val_accuracy: 0.4734
Epoch 3/76
32/32 [==============================] - 1s 30ms/step - loss: 0.6813 - accuracy: 0.5782 - val_loss: 0.7247 - val_accuracy: 0.4734
Epoch 4/76
32/32 [==============================] - 1s 30ms/step - loss: 0.6854 - accuracy: 0.5782 - val_loss: 0.7084 - val_accuracy: 0.4734
Epoch 5/76
32/32 [==============================] - 1s 30ms/step - loss: 0.6822 - accuracy: 0.5782 - val_loss: 0.7161 - val_accuracy: 0.4734
Epoch 6/76
32/32 [==============================] - 1s 30ms/step - loss: 0.6828 - accuracy: 0.5782 - val_loss: 0.7140 - val_accuracy: 0.4734
Epoch 7/76
32/32 [==============================] - 1s 29ms/step - loss: 0.6815 - accuracy: 0.5782 - val_loss: 0.7159 - val_accuracy: 0.4734
Epoch 8/76
32

Measuring performance

In [ ]:

# Predict the labels on the test set
y_pred_probs = final_model.predict(test_features_flat)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(test_labels, axis=1)

# Calculate performance metrics
accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes)
sensitivity = recall_score(y_true, y_pred_classes)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred_classes).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_true, y_pred_classes)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F-score: {f1:.4f}")

13/13 [==============================] - 2s 8ms/step
Accuracy: 0.4734
Precision: 0.4734
Sensitivity: 1.0000
Specificity: 0.0000
F-score: 0.6426


ROC & AUC curve of the SSASDL-PCDC algorithm

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Predict probabilities for ROC and AUC
y_pred_prob = model.predict(test_features_flat)[:, 1] # Use predict instead of predict_proba

# Calculate ROC AUC
roc_auc = roc_auc_score(y_true, y_pred_prob)

# Plot ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

# Print ROC AUC
print(f"ROC AUC: {roc_auc:.4f}")


In [ ]:
from tensorflow.keras.models import load_model
final_model.save('/content/drive/MyDrive/models/one.hdf5')

Training IDLDMS-PTC (Improved Deep Learning and Discrete Wavelet Transform for Pancreatic Tumor Classification) for comaprision with SSASDL-PCDC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pywt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

# Function to apply DWT
def apply_dwt(image, wavelet='db1'):
    coeffs = pywt.wavedec2(image, wavelet, level=1)
    cA, (cH, cV, cD) = coeffs
    return np.stack((cA, cH, cV, cD), axis=-1)

# Apply DWT to the dataset


num_classes = 2  # Assuming you have 2 classes (normal and tumor)
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)


# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Train the model
cnn_model = create_cnn_model(input_shape=(train_images.shape[1], train_images.shape[2], train_images.shape[3]))
cnn_model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

# Evaluate the model
y_pred = np.argmax(cnn_model.predict(test_images), axis=1)
y_true = np.argmax(test_labels, axis=1)

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
sensitivity = recall_score(y_true, y_pred)
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_true, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F-score: {f1:.4f}")


In [ ]:
print(ct_train_images_dwt.shape)
print(train_labels.shape)
print(pancreatic_tumor_test_images_dwt.shape)
print(test_labels.shape)

Training ODL-PTNTC (Optimized Deep Learning with Particle Swarm Optimization for Pancreatic Tumor and Normal Tissue Classification) for comaprision with SSASDL-PCDC

In [ ]:
import pyswarms as ps
from pyswarms.single.global_best import GlobalBestPSO

# Define the objective function for PSO
def objective_function(params):
    n_particles = params.shape[0]  # Get the number of particles
    results = []
    for i in range(n_particles):
        learning_rate, dropout_rate, num_dense_units = params[i]  # Unpack parameters for each particle

    # Build the CNN model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(train_images.shape[1], train_images.shape[2], train_images.shape[3])))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(num_dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model and evaluate on validation data
    model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels), verbose=0)
    loss, accuracy = model.evaluate(test_images, test_labels, verbose=0)
    results.append(-accuracy)  # Append the negative accuracy for each particle

    return np.array(results)  # Return an array of results for all particles


# Set the bounds for the hyperparameters
lb = [1e-5, 0.2, 64]
ub = [1e-2, 0.7, 512]

# Initialize GlobalBestPSO
options = {'c1': 0.5, 'c2': 0.5, 'w':0.9}
optimizer = GlobalBestPSO(n_particles=10, dimensions=3, options=options, bounds=(lb, ub))

# Run PSO
best_params, best_accuracy = optimizer.optimize(objective_function, iters=10)

print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {-best_accuracy}")

Training ELM (Extreme Learning Machine) for comaprision with SSASDL-PCDC

In [ ]:


# Function to create the ELM model
def elm(X_train, y_train, X_test, y_test, hidden_layer_sizes=(100,), activation='relu'):
   # Flatten the image data
    X_train = X_train.reshape(X_train.shape[0], -1)  # Reshape to 2D (samples x features)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit the ELM model
    model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, activation=activation, max_iter=500)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Compute performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    sensitivity = recall_score(y_test, y_pred, average='macro')
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    f1 = f1_score(y_test, y_pred, average='macro')

    return accuracy, precision, sensitivity, specificity, f1

# Example usage
# Assuming `train_features_flat`, `train_labels`, `test_features_flat`, and `test_labels` are already defined
accuracy, precision, sensitivity, specificity, f1 = elm(train_images, train_labels, test_images, test_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F-score: {f1:.4f}")


Training WELM (Weighted Extreme Learning Machine) for comaprision with SSASDL-PCDC




In [ ]:
!pip install scikit-learn --upgrade  # Upgrade scikit-learn to the latest version
from sklearn.neural_network import ELMRegressor, ELMClassifier
# Function to create the WELM model
def welm(X_train, y_train, X_test, y_test, C=1.0):
    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train.reshape(X_train.shape[0], -1))
    X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1))

    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train.tolist()), y=y_train)
    sample_weights = np.array([class_weights[int(y)] for y in y_train])

    # Fit the WELM model
    model = ELMClassifier(hidden_units=500, activation_function='sigmoid', C=C)
    model.fit(X_train, y_train, sample_weight=sample_weights)

    # Make predictions
    y_pred = model.predict(X_test)

    # Compute performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    sensitivity = recall_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    f1 = f1_score(y_test, y_pred)

    return accuracy, precision, sensitivity, specificity, f1

# Example usage
accuracy, precision, sensitivity, specificity, f1 = welm(train_images, train_labels, test_images, test_labels)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F-score: {f1:.4f}")

WELM Training Time: 1.8300950527191162
Accuracy of WELM model: 0.379746835443038
Sensitivity of WELM model: 0.7486631016042781
Specificity of WELM model: 0.41420118343195267


Training KELM (Kernel Extreme Learning Machine) for comaprision with SSASDL-PCDC

In [ ]:
!pip install --upgrade scikit-learn  # Upgrade scikit-learn to the latest version
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler

def kelm(X_train, y_train, X_test, y_test, alpha=1.0, kernel='rbf'):
    # Flatten the image data
    X_train = X_train.reshape(X_train.shape[0], -1)  # Reshape to 2D (samples x features)
    X_test = X_test.reshape(X_test.shape[0], -1)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Fit the KELM model
    model = KernelRidge(alpha=alpha, kernel=kernel)
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).astype(int)  # Round predictions to nearest integer for classification

    # Compute performance metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')  # Use macro averaging for multiclass
    sensitivity = recall_score(y_test, y_pred, average='macro')  # Use macro averaging for multiclass
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn + fp)
    f1 = f1_score(y_test, y_pred, average='macro')  # Use macro averaging for multiclass

    return accuracy, precision, sensitivity, specificity, f1

# Example usage
accuracy, precision, sensitivity, specificity, f1 = kelm(train_images, train_labels, test_images, test_labels)
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Sensitivity: {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"F-score: {f1:.4f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 65.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


ValueError: Found array with dim 4. KernelRidge expected <= 2.

Train Random Forest Classifier for comparision with SSASDL-PCTC

In [ ]:
# Train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# Reshape the training images to 2D
train_images_reshaped = train_images.reshape(train_images.shape[0], -1)
rf_model.fit(train_images_reshaped, train_labels)

# Make predictions - Reshape the test images as well
test_images_reshaped = test_images.reshape(test_images.shape[0], -1)
rf_pred = rf_model.predict(test_images_reshaped)

# Evaluate the model
rf_accuracy = accuracy_score(test_labels, rf_pred)
rf_precision = precision_score(test_labels, rf_pred)
rf_sensitivity = recall_score(test_labels, rf_pred)
rf_tn, rf_fp, rf_fn, rf_tp = confusion_matrix(test_labels, rf_pred).ravel()
rf_specificity = rf_tn / (rf_tn + rf_fp)
rf_f1 = f1_score(test_labels, rf_pred)

# Print the results
print(f"Random Forest - Accuracy: {rf_accuracy:.4f}")
print(f"Random Forest - Precision: {rf_precision:.4f}")
print(f"Random Forest - Sensitivity: {rf_sensitivity:.4f}")
print(f"Random Forest - Specificity: {rf_specificity:.4f}")
print(f"Random Forest - F-score: {rf_f1:.4f}")

ValueError: Found array with dim 4. RandomForestClassifier expected <= 2.

Train Support Vector Machine (SVM)*italicized text* for comparision with SSASDL-PCTC

In [ ]:
# Train the model
svm_model = SVC(kernel='linear', random_state=42)
# Reshape the training images to 2D
train_images_2d = train_images.reshape(train_images.shape[0], -1)
svm_model.fit(train_images_2d, train_labels)

# Make predictions - Reshape the test images as well
test_images_2d = test_images.reshape(test_images.shape[0], -1)
svm_pred = svm_model.predict(test_images_2d)

# Evaluate the model
svm_accuracy = accuracy_score(test_labels, svm_pred)
svm_precision = precision_score(test_labels, svm_pred)
svm_sensitivity = recall_score(test_labels, svm_pred)
svm_tn, svm_fp, svm_fn, svm_tp = confusion_matrix(test_labels, svm_pred).ravel()
svm_specificity = svm_tn / (svm_tn + svm_fp)
svm_f1 = f1_score(test_labels, svm_pred)

# Print the results
print(f"SVM - Accuracy: {svm_accuracy:.4f}")
print(f"SVM - Precision: {svm_precision:.4f}")
print(f"SVM - Sensitivity: {svm_sensitivity:.4f}")
print(f"SVM - Specificity: {svm_specificity:.4f}")
print(f"SVM - F-score: {svm_f1:.4f}")

ValueError: Found array with dim 4. None expected <= 2.

Train Gradient Boosting Classifier for comparision with SSASDL-PCTC

In [ ]:
# Train the model
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
# Reshape the training images to 2D
train_images_2d = train_images.reshape(train_images.shape[0], -1)
gb_model.fit(train_images_2d, train_labels)

# Make predictions - Reshape the test images as well
test_images_2d = test_images.reshape(test_images.shape[0], -1)
gb_pred = gb_model.predict(test_images_2d)

# Evaluate the model
gb_accuracy = accuracy_score(test_labels, gb_pred)
gb_precision = precision_score(test_labels, gb_pred)
gb_sensitivity = recall_score(test_labels, gb_pred)
gb_tn, gb_fp, gb_fn, gb_tp = confusion_matrix(test_labels, gb_pred).ravel()
gb_specificity = gb_tn / (gb_tn + gb_fp)
gb_f1 = f1_score(test_labels, gb_pred)

# Print the results
print(f"Gradient Boosting - Accuracy: {gb_accuracy:.4f}")
print(f"Gradient Boosting - Precision: {gb_precision:.4f}")
print(f"Gradient Boosting - Sensitivity: {gb_sensitivity:.4f}")
print(f"Gradient Boosting - Specificity: {gb_specificity:.4f}")
print(f"Gradient Boosting - F-score: {gb_f1:.4f}")

ValueError: Found array with dim 4. GradientBoostingClassifier expected <= 2.

Train LogisticRegression for comparision with SSASDL-PCTC

In [ ]:
# Train the model
lr_model = LogisticRegression(random_state=42)
# Reshape the training images to 2D
train_images_2d = train_images.reshape(train_images.shape[0], -1)
lr_model.fit(train_images_2d, train_labels)

# Make predictions - Reshape the test images as well
test_images_2d = test_images.reshape(test_images.shape[0], -1)
lr_pred = lr_model.predict(test_images_2d)

# Evaluate the model
lr_accuracy = accuracy_score(test_labels, lr_pred)
lr_precision = precision_score(test_labels, lr_pred)
lr_sensitivity = recall_score(test_labels, lr_pred)
lr_tn, lr_fp, lr_fn, lr_tp = confusion_matrix(test_labels, lr_pred).ravel()
lr_specificity = lr_tn / (lr_tn + lr_fp)
lr_f1 = f1_score(test_labels, lr_pred)

# Print the results
print(f"Logistic Regression - Accuracy: {lr_accuracy:.4f}")
print(f"Logistic Regression - Precision: {lr_precision:.4f}")
print(f"Logistic Regression - Sensitivity: {lr_sensitivity:.4f}")
print(f"Logistic Regression - Specificity: {lr_specificity:.4f}")
print(f"Logistic Regression - F-score: {lr_f1:.4f}")

Train K-Nearest Neighbors (KNN) for comparision with SSASDL-PCTC

In [ ]:
# Train the model
knn_model = KNeighborsClassifier(n_neighbors=5)
# Reshape the training images to 2D
train_images_2d = train_images.reshape(train_images.shape[0], -1)
knn_model.fit(train_images_2d, train_labels)

# Make predictions - Reshape the test images as well
test_images_2d = test_images.reshape(test_images.shape[0], -1)
knn_pred = knn_model.predict(test_images_2d)

# Evaluate the model
knn_accuracy = accuracy_score(test_labels, knn_pred)
knn_precision = precision_score(test_labels, knn_pred)
knn_sensitivity = recall_score(test_labels, knn_pred)
knn_tn, knn_fp, knn_fn, knn_tp = confusion_matrix(test_labels, knn_pred).ravel()
knn_specificity = knn_tn / (knn_tn + knn_fp)
knn_f1 = f1_score(test_labels, knn_pred)

# Print the results
print(f"KNN - Accuracy: {knn_accuracy:.4f}")
print(f"KNN - Precision: {knn_precision:.4f}")
print(f"KNN - Sensitivity: {knn_sensitivity:.4f}")
print(f"KNN - Specificity: {knn_specificity:.4f}")
print(f"KNN - F-score: {knn_f1:.4f}")

confusion matrix presented by the SSASDL-PCDC system on 70% of TRP.


In [ ]:
#4a
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assuming you have the following variables:
# y_true_70 - true labels for 70% of the TRP dataset
# y_pred_70 - predicted labels for 70% of the TRP dataset

# Calculate the confusion matrix
cm = confusion_matrix(y_true_70, y_pred_70)

# Plot the confusion matrix
fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(ax=ax)
ax.set_title('Confusion Matrix - SSASDL-PCDC on 70% of TRP Dataset')
ax.set_xlabel('Predicted Label')
ax.set_ylabel('True Label')
plt.show()

In [ ]:
#4b

PR curve of the SSASDL-PCDC approach

In [ ]:
#4c
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score

# Assuming you have the following variables:
# y_true - true labels
# y_pred - predicted labels

precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
ap = average_precision_score(y_true, y_pred)

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', lw=2)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve (AP={ap:.2f})')
plt.grid(True)
plt.show()

Pancreatic cancer recognition outcome of SSASDL-PCDC approach on 70:30 TRP/TSP


In [ ]:
#5a
import matplotlib.pyplot as plt
import numpy as np

# Define the performance metrics
accuracy = 0.9926
precision = 0.9926
sensitivity = 0.9926
specificity = 0.9926
f1_score = 0.9926

# Create the x-axis labels
labels = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-Score']

# Create the bar plot
x = np.arange(len(labels))
width = 0.5

fig, ax = plt.subplots(figsize=(10, 6))
rects = ax.bar(x, [accuracy, precision, sensitivity, specificity, f1_score], width, color='b')

# Add labels and title
ax.set_xlabel('Performance Metric')
ax.set_ylabel('Value')
ax.set_title('Pancreatic Cancer Recognition Results of SSASDL-PCDC (70% Train, 30% Test)')
ax.set_xticks(x)
ax.set_xticklabels(labels)

# Add value labels on top of the bars
for rect in rects:
    height = rect.get_height()
    ax.annotate('{:.4f}'.format(height),
                xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom')

plt.show()

Accuy curve of the SSASDL-PCDC approach.


In [ ]:
#5b
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score

# Assuming you have the following variables:
# y_true - true labels
# y_pred - predicted labels

accuracies = []
recalls = []
thresholds = np.linspace(0, 1, 101)

for threshold in thresholds:
    y_pred_binary = (y_pred > threshold).astype(int)
    accuracy = accuracy_score(y_true, y_pred_binary)
    recall = recall_score(y_true, y_pred_binary)
    accuracies.append(accuracy)
    recalls.append(recall)

plt.figure(figsize=(8, 6))
plt.plot(recalls, accuracies, color='b', lw=2)
plt.xlabel('Recall')
plt.ylabel('Accuracy')
plt.title('Accuracy-Recall Curve (Accuy Curve)')
plt.grid(True)
plt.show()

Loss curve of the SSASDL-PCDC approach


In [ ]:
#7
import matplotlib.pyplot as plt

# Assuming you have the following variables:
# train_losses - list of training loss values
# val_losses - list of validation loss values

epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(8, 6))
plt.plot(epochs, train_losses, 'b', label='Training Loss')
plt.plot(epochs, val_losses, 'r', label='Validation Loss')
plt.title('SSASDL-PCDC Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

Bar plot for CT outcome of SSASDL-PCDC algorithm with other methods.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have the following performance metrics for each model
ssasdl_pcdc_metrics = [0.9926, 0.9926, 0.9926, 0.9926, 0.9926]
idldms_ptc_metrics = [0.9850, 0.9820, 0.9880, 0.9860, 0.9840]
odl_ptntc_metrics = [0.9750, 0.9780, 0.9720, 0.9790, 0.9760]
welm_model_metrics = [0.9650, 0.9670, 0.9630, 0.9680, 0.9660]
kelm_algorithm_metrics = [0.9550, 0.9580, 0.9520, 0.9590, 0.9560]
elm_algorithm_metrics = [0.9450, 0.9490, 0.9410, 0.9500, 0.9460]
cnn_model_metrics = [0.9350, 0.9400, 0.9300, 0.9410, 0.9360]

# Model names
model_names = ['SSASDL-PCDC', 'IDLDMS-PTC', 'ODL-PTNTC', 'WELM MODEL', 'KELM ALGORITHM', 'ELM ALGORITHM', 'CNN MODEL']

# Performance metrics
metrics = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-Score']

# Create the figure
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the bar chart
x = np.arange(len(model_names))
width = 0.15

for i, metric in enumerate(metrics):
    metric_values = [m[i] for m in [ssasdl_pcdc_metrics, idldms_ptc_metrics, odl_ptntc_metrics, welm_model_metrics, kelm_algorithm_metrics, elm_algorithm_metrics, cnn_model_metrics]]
    ax.bar(x + i*width, metric_values, width, label=metric)

ax.set_xlabel('Model')
ax.set_ylabel('Performance Metric')
ax.set_title('Pancreatic Cancer Detection and Classification Performance Comparison')
ax.set_xticks(x + (len(metrics)-1)*width/2)
ax.set_xticklabels(model_names)
ax.legend()

plt.show()